In [1]:
import cv2, sys, pickle
import numpy as np
import dlib
from IPython.display import clear_output, display

In [2]:
# Convert the shape of 68 features into numpy array format
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

In [3]:
def load_pretrained_model():
    imagePath = './media/nba1.jpg'
    pkl_filename = './model/pickle_logist_model.pkl'
    faceCascade_filename = '/model/haarcascade_frontalface_default.xml'
    modelPath = './model/shape_predictor_68_face_landmarks.dat'
    
    # Load Pre-trained Engagement Model
    with open(pkl_filename, 'rb') as file:
        myLogsticModel = pickle.load(file)
        
    # Load Pre-trained Face detection Model
    # faceCascade = cv2.CascadeClassifier(faceCascade_filename)
    
    # Load Dlib Face Detector
    faceDetector = dlib.get_frontal_face_detector()
    
    # Load Pre-trained Landmark detection Model
    shapePredictor = dlib.shape_predictor(modelPath)
    
    
    return myLogsticModel, faceDetector, shapePredictor

In [4]:
def analyzeEngagement(image,engagement_model,face_model,landmark_model, noface_detect):
    # Load Models
    #engagement_model, face_model, landmark_model = load_pretrained_model()
    
    # Gray Image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect Faces
    # faces = face_model.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=3,minSize=(30, 30))
    # print(len(faces))
    height = gray.shape[0]
    frame_resize_scale = float(height)/480
    
    imageSmall= cv2.resize(gray, None, fx = 1.0/frame_resize_scale, fy = 1.0/frame_resize_scale, interpolation = cv2.INTER_LINEAR)
    faces = face_model(imageSmall,0)
    
    
    
    if len(faces) > 0:
        noface_detect = 0
        # print(f'{len(faces)} faces detected')
        
        for (i,face) in enumerate(faces):
            sub_face_area = dlib.rectangle(int(face.left() * frame_resize_scale),
                                           int(face.top() * frame_resize_scale),
                                           int(face.right() * frame_resize_scale),
                                           int(face.bottom() * frame_resize_scale))
            
            sub_face = gray[sub_face_area.top():sub_face_area.bottom(),sub_face_area.left():sub_face_area.right()]
            
            try:
                sub_face_resized = cv2.resize(sub_face,(224,224),interpolation = cv2.INTER_AREA)
                rect_value = dlib.rectangle(0, 0, 224, 224)

                landmarks = landmark_model(sub_face_resized, rect_value)

                landmarks = shape_to_np(landmarks)

                landmarks_flatten = landmarks.reshape(-1)

                landmarks_expanded = np.expand_dims(landmarks_flatten,axis=1).T

                face_prediction_result = engagement_model.predict(landmarks_expanded)[0]
                
                if face_prediction_result == 1:
                    enagement_eachface[i] = 0
                else:
                    enagement_eachface[i] += 1
               
                if face_prediction_result:
                    cv2.rectangle(image, (sub_face_area.left(), sub_face_area.top()), (sub_face_area.right(), sub_face_area.bottom()), (0,255,0), 2)
                    cv2.putText(image, f"Engaged", (sub_face_area.left(), sub_face_area.top() - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                else:
                    cv2.rectangle(image,  (sub_face_area.left(), sub_face_area.top()), (sub_face_area.right(), sub_face_area.bottom()), (0,0,255), 2)
                    cv2.putText(image, f"Not Engaged", (sub_face_area.left() - 5, sub_face_area.top() - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            except Exception as e:
                print(e)
    else:
        noface_detect += 1
        # print('no face detected, noface_detect = ',noface_detect)
    return image, noface_detect

In [5]:
# cap = cv2.VideoCapture(f'./media/1.mp4')

In [11]:
face1 = []
face2 = []
face3 = []

enagement_eachface = [0,0,0]
noface_detect = 0

engagement_model, face_model, landmark_model = load_pretrained_model()
framesSkipping = 1
count = 1


cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH,720)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
cap.set(10,150)

fps = cap.get(cv2.CAP_PROP_FPS)
print('fps = ',fps)
idle_time = 1 #seconds
idle_frame = idle_time * fps

while True:
    ret, image = cap.read()
    
    if not ret:
        break
    
#     if (count % framesSkipping == 0):
        
        
#     count += 1

    #clear_output(wait=True)    
    #print(f"count {count} % framesSkipping = {count%framesSkipping} ")

    image,res = analyzeEngagement(image,engagement_model,face_model,landmark_model, noface_detect)
    noface_detect = res
    
    clear_output(wait=True) 
    print('enagement_eachface = ',enagement_eachface)
    if noface_detect >= idle_frame:
        print(f'No face detect for {round(noface_detect/fps,1)} seconds.')
        cv2.putText(image, f"No face detect for {round(noface_detect/fps,1)} seconds.", 
                    (int(image.shape[1]/2) - 150, 15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    if enagement_eachface[0] >= idle_frame:
        print(f'Face #1 is not engaged for {round(enagement_eachface[0]/fps,1)} seconds.')
        cv2.putText(image, f"Face #1 is not engaged for {round(enagement_eachface[0]/fps,1)} seconds.", 
                    (int(image.shape[1]/2) - 150, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    if enagement_eachface[1] >= idle_frame:
        print(f'Face #2 is not engaged for {round(enagement_eachface[1]/fps,1)} seconds.')
        cv2.putText(image, f"Face #2 is not engaged for {round(enagement_eachface[1]/fps,1)} seconds.", 
                    (int(image.shape[1]/2) - 150, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    if enagement_eachface[2] >= idle_frame:
        print(f'Face #3 is not engaged for {round(enagement_eachface[2]/fps,1)} seconds.')
        cv2.putText(image, f"Face #3 is not engaged for {round(enagement_eachface[2]/fps,1)} seconds.", 
                    (int(image.shape[1]/2) - 150, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    cv2.imshow('Image', image)
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break 
        
cap.release()
cv2.destroyAllWindows()

enagement_eachface =  [0, 0, 0]
